# Merging into Final Friction Surface(s)
This notebook merges the on road and off road friction surfaces to arrive at multimodal friction surfaces

In [ ]:
import os, sys
os.environ['USE_PYGEOS'] = '0'

import common_rasterio_ops as rast_ops

import numpy as np
from numpy import pi, log, tan, empty, float32, arctan, rad2deg, gradient
from numpy import arctan2, reshape, where
from scipy.ndimage import gaussian_gradient_magnitude

import rasterio
from rasterio import features, transform
from rasterio.mask import mask
from rasterio.transform import Affine
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.io import MemoryFile

import pandas as pd
import geopandas as gpd

import shapely
from shapely.geometry import shape, box, Polygon

import json

### Setup

In [ ]:
data_root = 'D:\\github_test\\'

##################################################################
##################################################################
#read project input parameters that will eventually be passed from the UI
data_file = data_root + 'project_data.json'

##################################################################
##################################################################
#read project variables that will come from UI so that we have our parameters and file locations
with open(data_file, 'rb') as f:
    data_loaded = json.load(f)
f.close()

##################################################################
##################################################################
#read information from the project setup file that's relevant to this section of code
#imports
local_dem_folder = data_loaded['local_dem_folder']
local_lc_folder = data_loaded['local_lc_folder']
local_boundaries_folder = data_loaded['local_boundaries_folder']
fric_dir = data_loaded['fric_dir']
fric_dir_onroad = data_loaded['fric_dir_onroad']
dest_crs = data_loaded['dest_crs']
dest_crs_id = data_loaded['dest_crs_id']
buffer_m = data_loaded['buffer_m']
level = data_loaded['level']
if level != 'custom':
    shapefile_adm_field = data_loaded['shapefile_adm_field']
    adm_name = data_loaded['adm_name']

##################################################################
##################################################################
seasons = sorted([os.path.join(local_lc_folder,file) \
            for file \
            in os.listdir(local_lc_folder) \
            if file.endswith(".tif")])

for strnum in range(0, len(seasons)):
    seasons[strnum] = str.replace(seasons[strnum], local_lc_folder,"")
    seasons[strnum] = str.replace(seasons[strnum], ".tif","")   

# Create final multi-modal friction surface

Now that the walking surface is created, we can combine it with the previously created on-road speed surfaces, once we transform them to friction surfaces. Combining the two yields the final multi-modal friction surface

In [ ]:
for season_num in range(0, len(seasons)):
    
    current_season = seasons[season_num]
    
    # Load in the walking speed surface.
    with rasterio.open(os.path.join(fric_dir,current_season+'_walk.tif')) as offr_src:
        friction_walk_final = offr_src.read()
        offr_profile = offr_src.profile
        offr_transform = offr_src.transform


    # Load in the on-road speed surface.
    with rasterio.open(os.path.join(fric_dir_onroad,current_season+'_onroad.tif')) as onr_src:
        drivespeed_array = onr_src.read()
        onr_profile = onr_src.profile
        onr_transform = onr_src.transform

    # Compute a final friction surface by taking the lowest (speediest) value from each raster. In almost all cases this will be the driving surface speed. Note that because our NoData value is `-99999` it will always be lowest, so we need to exclude this
    # friction_final = np.where(((friction_drive_step2 < friction_walk_final) | (friction_walk_final == -99999)), friction_drive_step2, friction_walk_final)
    friction_final = np.where(((drivespeed_array < friction_walk_final) | (friction_walk_final == -99999)), drivespeed_array, friction_walk_final)

    ### Adjustments to reduce final file size
    # Round up to 8 decimal points to reduce file size. Note this slightly reduces accuracy

    friction_final = np.round(friction_final,8).astype(np.float32)

    ### Remove index band
    friction_final = friction_final[0, :, :]

    ### Export
    # Modify the output profile to compress efficiently
    export_profile = offr_profile.copy()
    export_profile.update({"dtype":'float32',\
                           "COMPRESS":'ZSTD',
                           "NUM_THREADS":'ALL_CPUS',
                           "nodata":-99999})

    with rasterio.open(os.path.join(fric_dir,current_season+'_multi.tif'),'w',**export_profile) as dst:
        dst.write(friction_final,indexes=1)
        dst.build_overviews = ([2,4,8,10,14,16],Resampling.nearest) # build pyramids for quick viewing in desktop GIS software
        
# del vert_dist_switchback
del friction_final,drivespeed_array,friction_walk_final